# Redes neurais recorrentes

No módulo anterior, abordamos representações semânticas ricas de texto. A arquitetura que temos usado captura o significado agregado das palavras em uma sentença, mas não leva em conta a **ordem** das palavras, porque a operação de agregação que segue as embeddings remove essa informação do texto original. Como esses modelos não conseguem representar a ordem das palavras, eles não podem resolver tarefas mais complexas ou ambíguas, como geração de texto ou resposta a perguntas.

Para capturar o significado de uma sequência de texto, usaremos uma arquitetura de rede neural chamada **rede neural recorrente**, ou RNN. Ao usar uma RNN, passamos nossa sentença pela rede um token de cada vez, e a rede produz algum **estado**, que então passamos novamente para a rede junto com o próximo token.

![Imagem mostrando um exemplo de geração de rede neural recorrente.](../../../../../translated_images/pt-BR/rnn.27f5c29c53d727b5.webp)

Dada a sequência de entrada de tokens $X_0,\dots,X_n$, a RNN cria uma sequência de blocos de rede neural e treina essa sequência de ponta a ponta usando retropropagação. Cada bloco de rede recebe um par $(X_i,S_i)$ como entrada e produz $S_{i+1}$ como resultado. O estado final $S_n$ ou a saída $Y_n$ é enviado para um classificador linear para produzir o resultado. Todos os blocos de rede compartilham os mesmos pesos e são treinados de ponta a ponta usando uma única passagem de retropropagação.

> A figura acima mostra a rede neural recorrente na forma expandida (à esquerda) e em uma representação recorrente mais compacta (à direita). É importante perceber que todas as células RNN têm os mesmos **pesos compartilháveis**.

Como os vetores de estado $S_0,\dots,S_n$ são passados pela rede, a RNN é capaz de aprender dependências sequenciais entre palavras. Por exemplo, quando a palavra *não* aparece em algum lugar da sequência, ela pode aprender a negar certos elementos dentro do vetor de estado.

Internamente, cada célula RNN contém duas matrizes de peso: $W_H$ e $W_I$, e um viés $b$. Em cada etapa da RNN, dado o input $X_i$ e o estado de entrada $S_i$, o estado de saída é calculado como $S_{i+1} = f(W_H\times S_i + W_I\times X_i+b)$, onde $f$ é uma função de ativação (frequentemente $\tanh$).

> Para problemas como geração de texto (que abordaremos na próxima unidade) ou tradução automática, também queremos obter algum valor de saída em cada etapa da RNN. Nesse caso, há também outra matriz $W_O$, e a saída é calculada como $Y_i=f(W_O\times S_i+b_O)$.

Vamos ver como redes neurais recorrentes podem nos ajudar a classificar nosso conjunto de dados de notícias.

> Para o ambiente de sandbox, precisamos executar a célula a seguir para garantir que a biblioteca necessária esteja instalada e os dados sejam pré-carregados. Se você estiver executando localmente, pode ignorar a célula a seguir.


In [1]:
import sys
!{sys.executable} -m pip install --quiet tensorflow_datasets==4.4.0
!cd ~ && wget -q -O - https://mslearntensorflowlp.blob.core.windows.net/data/tfds-ag-news.tgz | tar xz

In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

# We are going to be training pretty large models. In order not to face errors, we need
# to set tensorflow option to grow GPU memory allocation when required
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

ds_train, ds_test = tfds.load('ag_news_subset').values()

Ao treinar modelos grandes, a alocação de memória da GPU pode se tornar um problema. Também pode ser necessário experimentar diferentes tamanhos de minibatch, para que os dados caibam na memória da GPU e o treinamento seja rápido o suficiente. Se você estiver executando este código em sua própria máquina com GPU, pode experimentar ajustar o tamanho do minibatch para acelerar o treinamento.

> **Note**: Certas versões de drivers da NVidia são conhecidas por não liberar a memória após o treinamento do modelo. Estamos executando vários exemplos neste notebook, e isso pode causar o esgotamento da memória em algumas configurações, especialmente se você estiver realizando seus próprios experimentos como parte do mesmo notebook. Se você encontrar erros estranhos ao iniciar o treinamento do modelo, pode ser necessário reiniciar o kernel do notebook.


In [3]:
batch_size = 16
embed_size = 64

## Classificador RNN simples

No caso de uma RNN simples, cada unidade recorrente é uma rede linear simples, que recebe um vetor de entrada e um vetor de estado, e produz um novo vetor de estado. No Keras, isso pode ser representado pela camada `SimpleRNN`.

Embora possamos passar tokens codificados em one-hot diretamente para a camada RNN, isso não é uma boa ideia devido à sua alta dimensionalidade. Por isso, utilizaremos uma camada de embedding para reduzir a dimensionalidade dos vetores de palavras, seguida por uma camada RNN e, por fim, um classificador `Dense`.

> **Note**: Em casos onde a dimensionalidade não é tão alta, por exemplo, ao usar tokenização a nível de caracteres, pode fazer sentido passar tokens codificados em one-hot diretamente para a célula RNN.


In [4]:
vocab_size = 20000

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=vocab_size,
    input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 64)          1280000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 16)                1296      
_________________________________________________________________
dense (Dense)                (None, 4)                 68        
Total params: 1,281,364
Trainable params: 1,281,364
Non-trainable params: 0
_________________________________________________________________


> **Nota:** Aqui usamos uma camada de embedding não treinada para simplificar, mas para obter melhores resultados, podemos usar uma camada de embedding pré-treinada com Word2Vec, como descrito na unidade anterior. Seria um bom exercício para você adaptar este código para funcionar com embeddings pré-treinados.

Agora vamos treinar nossa RNN. RNNs, em geral, são bastante difíceis de treinar, porque, uma vez que as células da RNN são desenroladas ao longo do comprimento da sequência, o número resultante de camadas envolvidas na retropropagação é bastante grande. Por isso, precisamos selecionar uma taxa de aprendizado menor e treinar a rede em um conjunto de dados maior para obter bons resultados. Isso pode levar bastante tempo, então é preferível usar uma GPU.

Para acelerar o processo, treinaremos o modelo RNN apenas nos títulos das notícias, omitindo a descrição. Você pode tentar treinar com a descrição e ver se consegue fazer o modelo treinar.


In [5]:
def extract_title(x):
    return x['title']

def tupelize_title(x):
    return (extract_title(x),x['label'])

print('Training vectorizer')
vectorizer.adapt(ds_train.take(2000).map(extract_title))

Training vectorizer


In [6]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize_title).batch(batch_size),validation_data=ds_test.map(tupelize_title).batch(batch_size))

7500/7500 [==============================] - 82s 11ms/step - loss: 0.6629 - acc: 0.7623 - val_loss: 0.5559 - val_acc: 0.7995


> **Nota** que a precisão provavelmente será menor aqui, porque estamos treinando apenas em títulos de notícias.


## Revisitando sequências de variáveis

Lembre-se de que a camada `TextVectorization` irá automaticamente preencher sequências de comprimento variável em um minibatch com tokens de preenchimento (pad tokens). Acontece que esses tokens também participam do treinamento, e isso pode complicar a convergência do modelo.

Existem várias abordagens que podemos adotar para minimizar a quantidade de preenchimento. Uma delas é reordenar o conjunto de dados pelo comprimento das sequências e agrupar todas as sequências por tamanho. Isso pode ser feito usando a função `tf.data.experimental.bucket_by_sequence_length` (veja a [documentação](https://www.tensorflow.org/api_docs/python/tf/data/experimental/bucket_by_sequence_length)).

Outra abordagem é usar **masking**. No Keras, algumas camadas suportam entradas adicionais que indicam quais tokens devem ser considerados durante o treinamento. Para incorporar masking em nosso modelo, podemos incluir uma camada `Masking` separada ([documentação](https://keras.io/api/layers/core_layers/masking/)) ou especificar o parâmetro `mask_zero=True` na nossa camada `Embedding`.

> **Note**: Este treinamento levará cerca de 5 minutos para completar uma época em todo o conjunto de dados. Sinta-se à vontade para interromper o treinamento a qualquer momento se perder a paciência. Outra opção é limitar a quantidade de dados usados para o treinamento, adicionando a cláusula `.take(...)` após os conjuntos de dados `ds_train` e `ds_test`.


In [7]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size,embed_size,mask_zero=True),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

7500/7500 [==============================] - 371s 49ms/step - loss: 0.5401 - acc: 0.8079 - val_loss: 0.3780 - val_acc: 0.8822


Agora que estamos utilizando mascaramento, podemos treinar o modelo em todo o conjunto de dados de títulos e descrições.

> **Note**: Você percebeu que estamos usando um vetorizador treinado nos títulos das notícias, e não no corpo completo do artigo? Isso pode fazer com que alguns tokens sejam ignorados, então é melhor re-treinar o vetorizador. No entanto, isso provavelmente terá um efeito muito pequeno, então vamos continuar utilizando o vetorizador pré-treinado anterior para manter a simplicidade.


## LSTM: Memória de longo e curto prazo

Um dos principais problemas das RNNs é o **desvanecimento de gradientes**. As RNNs podem ser bastante longas e podem ter dificuldade em propagar os gradientes até a primeira camada da rede durante a retropropagação. Quando isso acontece, a rede não consegue aprender relações entre tokens distantes. Uma maneira de evitar esse problema é introduzir **gerenciamento explícito de estado** usando **portas**. As duas arquiteturas mais comuns que introduzem portas são **memória de longo e curto prazo** (LSTM) e **unidade de retransmissão com portas** (GRU). Vamos abordar as LSTMs aqui.

![Imagem mostrando um exemplo de célula de memória de longo e curto prazo](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

Uma rede LSTM é organizada de maneira semelhante a uma RNN, mas há dois estados que são passados de camada para camada: o estado real $c$ e o vetor oculto $h$. Em cada unidade, o vetor oculto $h_{t-1}$ é combinado com a entrada $x_t$, e juntos eles controlam o que acontece com o estado $c_t$ e a saída $h_{t}$ por meio de **portas**. Cada porta tem ativação sigmoide (saída no intervalo $[0,1]$), que pode ser vista como uma máscara binária quando multiplicada pelo vetor de estado. As LSTMs possuem as seguintes portas (da esquerda para a direita na imagem acima):
* **porta de esquecimento**, que determina quais componentes do vetor $c_{t-1}$ precisamos esquecer e quais passar adiante.
* **porta de entrada**, que determina quanta informação do vetor de entrada e do vetor oculto anterior deve ser incorporada ao vetor de estado.
* **porta de saída**, que pega o novo vetor de estado e decide quais de seus componentes serão usados para produzir o novo vetor oculto $h_t$.

Os componentes do estado $c$ podem ser vistos como indicadores que podem ser ativados ou desativados. Por exemplo, quando encontramos o nome *Alice* na sequência, deduzimos que se refere a uma mulher e ativamos o indicador no estado que diz que temos um substantivo feminino na frase. Quando encontramos posteriormente as palavras *e Tom*, ativamos o indicador que diz que temos um substantivo no plural. Assim, manipulando o estado, podemos acompanhar as propriedades gramaticais da frase.

> **Note**: Aqui está um ótimo recurso para entender os detalhes internos das LSTMs: [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) por Christopher Olah.

Embora a estrutura interna de uma célula LSTM possa parecer complexa, o Keras oculta essa implementação dentro da camada `LSTM`, então a única coisa que precisamos fazer no exemplo acima é substituir a camada recorrente:


In [8]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.LSTM(8),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(8),validation_data=ds_test.map(tupelize).batch(8))

15000/15000 [==============================] - 188s 13ms/step - loss: 0.5692 - acc: 0.7916 - val_loss: 0.3441 - val_acc: 0.8870


## RNNs Bidirecionais e Multicamadas

Nos nossos exemplos até agora, as redes recorrentes operam do início de uma sequência até o final. Isso nos parece natural porque segue a mesma direção em que lemos ou ouvimos uma fala. No entanto, para cenários que exigem acesso aleatório à sequência de entrada, faz mais sentido executar o cálculo recorrente em ambas as direções. RNNs que permitem cálculos em ambas as direções são chamadas de **RNNs bidirecionais**, e podem ser criadas envolvendo a camada recorrente com uma camada especial chamada `Bidirectional`.

> **Note**: A camada `Bidirectional` faz duas cópias da camada dentro dela e define a propriedade `go_backwards` de uma dessas cópias como `True`, fazendo com que ela percorra a sequência na direção oposta.

Redes recorrentes, sejam unidirecionais ou bidirecionais, capturam padrões dentro de uma sequência e os armazenam em vetores de estado ou os retornam como saída. Assim como nas redes convolucionais, podemos construir outra camada recorrente após a primeira para capturar padrões de nível mais alto, construídos a partir de padrões de nível mais baixo extraídos pela primeira camada. Isso nos leva à noção de uma **RNN multicamada**, que consiste em duas ou mais redes recorrentes, onde a saída da camada anterior é passada para a próxima camada como entrada.

![Imagem mostrando uma RNN multicamada de memória de longo e curto prazo](../../../../../translated_images/pt-BR/multi-layer-lstm.dd975e29bb2a59fe.webp)

*Imagem retirada [deste post incrível](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) por Fernando López.*

O Keras torna a construção dessas redes uma tarefa fácil, porque você só precisa adicionar mais camadas recorrentes ao modelo. Para todas as camadas, exceto a última, precisamos especificar o parâmetro `return_sequences=True`, porque precisamos que a camada retorne todos os estados intermediários, e não apenas o estado final do cálculo recorrente.

Vamos construir uma LSTM bidirecional de duas camadas para o nosso problema de classificação.

> **Note** este código novamente leva bastante tempo para ser executado, mas nos dá a maior precisão que vimos até agora. Então talvez valha a pena esperar e ver o resultado.


In [9]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, 128, mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64,return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),    
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

5045/7500 [===================>..........] - ETA: 2:33 - loss: 0.3709 - acc: 0.8706

## RNNs para outras tarefas

Até agora, nos concentramos em usar RNNs para classificar sequências de texto. Mas elas podem lidar com muitas outras tarefas, como geração de texto e tradução automática — abordaremos essas tarefas na próxima unidade.



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automatizadas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte autoritativa. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações equivocadas decorrentes do uso desta tradução.
